<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Altsorgular(Subqueries)" data-toc-modified-id="Altsorgular(Subqueries)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Altsorgular(Subqueries)</a></span><ul class="toc-item"><li><span><a href="#Any-ve-All-Operatörleri" data-toc-modified-id="Any-ve-All-Operatörleri-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Any ve All Operatörleri</a></span><ul class="toc-item"><li><span><a href="#ANY-Operatörü" data-toc-modified-id="ANY-Operatörü-1.1.1"><span class="toc-item-num">1.1.1&nbsp;&nbsp;</span>ANY Operatörü</a></span></li><li><span><a href="#ALL-Operatörü" data-toc-modified-id="ALL-Operatörü-1.1.2"><span class="toc-item-num">1.1.2&nbsp;&nbsp;</span>ALL Operatörü</a></span></li></ul></li><li><span><a href="#Alt-Sorgular-ve-JOIN-Kullanımı" data-toc-modified-id="Alt-Sorgular-ve-JOIN-Kullanımı-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Alt Sorgular ve JOIN Kullanımı</a></span><ul class="toc-item"><li><span><a href="#GENEL-TEKRAR" data-toc-modified-id="GENEL-TEKRAR-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>GENEL TEKRAR</a></span></li></ul></li></ul></li></ul></div>

# Altsorgular(Subqueries)
Bir sorgu içerisinde, o sorgunun ihtiyaç duyduğu veri veya verileri getiren sorgulardır.

__Alt Sorgu Kullanımı__  
bookstore veritabanında "Gülün Adı" isimli kitabımızın sayfa sayısı 466 dır. Bu kitaptan daha fazla sayıda sayfası bulunan kitapları aşağıdaki sorgu yardımıyla sıralayabiliriz.   
```
SELECT *
FROM book
WHERE page_number > 466;
```
Ancak yukarıdaki sorgumuzda şöyle bir sorun var. 466 ifade statik bir ifade ve biz bu değeri bilmiyor olabiliriz. Bu nedenle buradaki 466 rakamını aşağıdaki sorgu yardımıyla bulabiliriz.
```
SELECT page_number
FROM book
WHERE title = 'Gülün Adı'
```
İşte bu ikinci sorgumuz ilk sorgumuzda bir alt sorgu görevi görebilir. Her iki sorguyu da birleştirelim.
```
SELECT *
FROM book
WHERE page_number >
(
SELECT page_number
FROM book
WHERE title = 'Gülün Adı'
);
```

Soyle daha fazla alt sorgu ile islem yapalim

```
SELECT title , page_number , (SELECT (MAX(page_number) FROM book) , ((SELECT (MAX(page_number) FROM book) - page_number ) AS difference
FROM book
WHERE page_number >
(
SELECT page_number FROM book
WHERE title = 'Gulun Adi'
(;
```
Yukaridaki sorgumuzda book tablosundan title ve page number i aldik ve ek olarak Max sayfa sayisi olan kitabi yanina bir sutun olarak ekledik , diger sutuna ise ilk sorgudan gelen page numberi maximum page number olan kitaptan cikartarak ek bir 'fark' sutunu ekledik.A na sorgumuzdaki page_number'i bir sayi ile degilde, bir alt sorgu ile tekrardan Gulun adi kitabindan fazla olacak sekilde gelmesi icin kosullandirdik.

## Any ve All Operatörleri
Any ve All operatörleri alt sorugularda sıklıkla kullanılır ve tek bir sütunda bulunan bir değerle bir değer dizisinin karşılaştırılmasını sağlar.

### ANY Operatörü
Alt sorgudan gelen herhangi bir değer koşulu sağlaması durumunda TRUE olarak ilgili değerin koşu sağlamasını sağlar.

__ORNEK KULLANIM__
```
SELECT first_name, last_name
FROM author
WHERE id = ANY
(
  SELECT id
  FROM book
  WHERE title = 'Abe Lincoln in Illinois' OR title = 'Saving Shiloh'
);
```

Yukarıda görmüş olduğunuz gibi alt sorgudan gelebilecek potansiye iki id değeri var, bu id değerinin her ikisi de birbirinden bağımsız olarak ana sorgudaki id sütununda bulunan değerler ile eşleştiği için sorgu sonucunda oluşan sana tabloda id değeri 4 ve 5 olan yazarlara ait first_name ve last_name değerlerini göreceğiz.

### ALL Operatörü

Alt sorgudan gelen tüm değerlerin koşulu sağlaması durumunda TRUE olarak döner.

bookstore veritabanındaki yine aynı sorguyu inceleyelim.

```
SELECT first_name, last_name
FROM author
WHERE id = ALL
(
  SELECT id
  FROM book
  WHERE title = 'Abe Lincoln in Illinois' OR title = 'Saving Shiloh'
);
```
Burada ne söylemiştik alt sorgu tarafından 4 ve 5 id leri gelecek burada eştlik olduğu için aynı anda 4 ve 5 in bu koşulu sağlaması olanaksız olduğu için herhangi bir değer dönülmeyecektir.

## Alt Sorgular ve JOIN Kullanımı
__İlk senaryomuz:__ bookstore veri tabanında kitap sayfası sayısı ortalama kitap sayfası sayısından fazla olan kitapların isimlerini, bu kitapların yazarlarına ait isim ve soyisim bilgileriyle birlikte sıralayınız.
```
SELECT author.first_name ,author.last_name , book.title ,book.page_number
FROM author
JOIN book
ON book.author_id = author.id
WHERE page_number >
(
    SELECT AVG(page_number)
    FROM book

);
```

Yukarıdaki sorgumuzda kitaplara ait yazar bilgilerini JOIN kullanarak elde ediyoruz. Ortalama sayfa sayısını da alt sorgudan getiriyoruz.

__Ikinci senaryomuz:__ dvdrental veritabanında en uzun filmlerin isimlerini aktör isim ve soyisimleriyle birlikte sıralayalım.

```
SELECT actor.first_name , actor.last_name ,film.title ,film.length
FROM actor
JOIN film_actor ON actor.actor_id = film_actor.actor_id
JOIN film ON film.film_id = film_actor.film_id
WHERE film.length =
(
    SELECT MAX(length) FROM film
);
```

Burada da görmüş olduğumuz gibi film lerin aktör bilgilerini ikili JOIN yapısı kullanarak elde ediyoruz. En uzun film süresini de alt sorgudan getiriyoruz.

### GENEL TEKRAR

__film tablosunda isminde en az 4 adet 'e' veya 'E' karakteri bulunan kç tane film vardır?__
```
SELECT COUNT(*)
FROM film
WHERE title ILIKE  '%e%e%e%e%'
```

__category tablosundan kategori isimlerini ve kategori başına düşen film sayılarını sıralayınız.__

```
SELECT COUNT(*),category.name
FROM category
JOIN film_category
ON film_category.category_id = category.category_id
JOIN film
ON film.film_id = film_category.film_id
GROUP BY category.name;
```
__film tablosunda içerisinden en fazla sayıda film bulunduran rating kategorisi hangisidir?__

```
SELECT COUNT(*) , rating
FROM film
GROUP BY rating
ORDER BY count DESC
LIMIT 1;
```

__film tablosundan 'K' karakteri ile başlayan en uzun ve replacenet_cost u en düşük 4 filmi sıralayınız.__

```
SELECT length,title,replacement_cost 
FROM film
WHERE title LIKE 'K%'
ORDER BY length DESC , replacement_cost ASC
LIMIT 3;
```